Используя датасет "Собаки и кошки":
1. Проведена аугментация изображений.
2. В качестве предобученной модели взята `MobileNet`
3. Создана модель многоклассовой классификации, приведенная ниже.
4. Результат првоерен на тестовой выборке.

In [ ]:
# from keras.applications import MobileNet
# from keras import models
# from keras.layers import GlobalAveragePooling2D, Dense, Dropout
# from keras import optimizers

# def model_maker():
#     base_model = MobileNet(include_top=False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))

#     for layer in base_model.layers[:]:
#         layer.trainable = False

#     input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
#     custom_model = base_model(input)
#     custom_model = GlobalAveragePooling2D()(custom_model)
#     custom_model = Dense(64, activation='relu')(custom_model)
#     custom_model = Dropout(0.5)(custom_model)
#     predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)

#     return Model(inputs=input, outputs=predictions)

In [ ]:
# model_maker.summary()

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip

!unzip -q -o "cat-and-dog" -d ./temp

--2024-11-03 22:32:38--  https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228082266 (218M) [application/x-zip-compressed]
Saving to: ‘cat-and-dog.zip’

cat-and-dog.zip     100%[===================>] 217.52M  29.6MB/s    in 8.5s    

2024-11-03 22:32:48 (25.7 MB/s) - ‘cat-and-dog.zip’ saved [228082266/228082266]



In [ ]:
from keras.applications import MobileNet
conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [ ]:
import os
import shutil

from keras import layers
from keras import models
from keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_IMG_PATH = './temp/training_set/training_set/'
TEST_IMG_PATH = './temp/test_set/test_set/'
BASE_DIR = './dataset/'

CLASS_LIST = sorted(os.listdir(TRAIN_IMG_PATH))
CLASS_COUNT = len(CLASS_LIST)

if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)

os.mkdir(BASE_DIR)

train_dir = os.path.join(BASE_DIR, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(BASE_DIR, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(BASE_DIR, 'test')
os.mkdir(test_dir)

def create_dataset(
    img_path: str,
    new_path: str,
    class_name: str,
    start_index: int,
    end_index: int

):

    src_path = os.path.join(img_path, class_name)
    dst_path = os.path.join(new_path, class_name)

    class_files = os.listdir(src_path)
    os.mkdir(dst_path)

    for fname in class_files[start_index : end_index]:
        src = os.path.join(src_path, fname)
        dst = os.path.join(dst_path, fname)
        shutil.copyfile(src, dst)


for class_label in range(CLASS_COUNT):
    class_name = CLASS_LIST[class_label]
    create_dataset(TRAIN_IMG_PATH, train_dir, class_name, 0, 3000)
    create_dataset(TRAIN_IMG_PATH, validation_dir, class_name, 3000, 4000)
    create_dataset(TEST_IMG_PATH, test_dir, class_name, 0, 1000)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 10

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 224, 224, 3))
    labels = np.zeros(shape=(sample_count, 2))

    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = inputs_batch

        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1

        if i * batch_size >= sample_count:
            break

    return features, labels

train_features, train_labels = extract_features(train_dir, 6000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 6000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
from keras.applications import MobileNet
from keras import models
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from keras import optimizers
from keras import losses

def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (224, 224, 3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(224, 224, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(2, activation='softmax')(custom_model)

    return models.Model(inputs=input, outputs=predictions)

In [ ]:
model = model_maker()

model.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=losses.categorical_crossentropy,
    metrics=['acc']
)

In [ ]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_acc', patience=3)

history = model.fit(
    train_features, train_labels,
    epochs=30,
    batch_size=32,
    validation_data=(validation_features, validation_labels),
    callbacks=[callback],
    verbose=1
)

Epoch 1/30
188/188 [==============================] - 36s 182ms/step - loss: 0.2262 - acc: 0.9050 - val_loss: 0.0869 - val_acc: 0.9690
Epoch 2/30
188/188 [==============================] - 32s 173ms/step - loss: 0.0925 - acc: 0.9637 - val_loss: 0.0711 - val_acc: 0.9700
Epoch 3/30
188/188 [==============================] - 32s 171ms/step - loss: 0.0724 - acc: 0.9707 - val_loss: 0.0677 - val_acc: 0.9690
Epoch 4/30
188/188 [==============================] - 34s 183ms/step - loss: 0.0609 - acc: 0.9768 - val_loss: 0.0624 - val_acc: 0.9750
Epoch 5/30
188/188 [==============================] - 32s 171ms/step - loss: 0.0539 - acc: 0.9800 - val_loss: 0.0612 - val_acc: 0.9740
Epoch 6/30
188/188 [==============================] - 31s 167ms/step - loss: 0.0469 - acc: 0.9815 - val_loss: 0.0610 - val_acc: 0.9750
Epoch 7/30
188/188 [==============================] - 30s 162ms/step - loss: 0.0446 - acc: 0.9835 - val_loss: 0.0589 - val_acc: 0.9760
Epoch 8/30
188/188 [==============================] - 3

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('Контрольная точность', test_acc)

Found 2000 images belonging to 2 classes.
50/50 [==============================] - 3s 43ms/step - loss: 0.0556 - acc: 0.9800
Контрольная точность 0.9800000190734863


In [ ]:
model.save('cats_dogs_mode.hdf5r')

In [ ]:
!zip -r /content/cats_dogs_mode.zip /content/cats_dogs_mode.hdf5r

  adding: content/cats_dogs_mode.hdf5r/ (stored 0%)
  adding: content/cats_dogs_mode.hdf5r/fingerprint.pb (stored 0%)
  adding: content/cats_dogs_mode.hdf5r/saved_model.pb (deflated 91%)
  adding: content/cats_dogs_mode.hdf5r/variables/ (stored 0%)
  adding: content/cats_dogs_mode.hdf5r/variables/variables.index (deflated 75%)
  adding: content/cats_dogs_mode.hdf5r/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/cats_dogs_mode.hdf5r/keras_metadata.pb (deflated 96%)
  adding: content/cats_dogs_mode.hdf5r/assets/ (stored 0%)


In [ ]:
from google.colab import files
files.download("/content/cats_dogs_mode.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>